In [27]:
import pandas as pd
import numpy as np
import dateutil

$\space$
# Label Set --> year | DOY 

In [72]:
# 1921-2016 -- Washington.D.C
WDC_df = pd.read_csv('WashingtonDC_1921-2016_labels.csv', header=6, usecols=['Year','Yoshino peak bloom date']).fillna(0)
WDC_df.to_csv("0_Labels_WDC.csv", header=['year','DOY'], index=False)          

# 1881-2015 -- Kyoto
df = pd.read_excel('Kyoto_AD 801-2015_labels.xls', header = 25)
kyoto_df = df.loc[1080:, ['AD', 'Full-flowering date (DOY)']].fillna(0)
kyoto_df["Full-flowering date (DOY)"] = kyoto_df["Full-flowering date (DOY)"].astype(int)
kyoto_df.to_csv("0_Labels_Kyoto.csv", header=['year','DOY'], index=False)     

$\space$
# Temperature Set --> year | month | day | DOY | TAVG | TMAX | TMIN 

### 日本 1881-2020 的日均温度数据(日本温度记录元年是1881/1/1)

In [24]:
files = ['data_1881-1891.csv', 'data_1892-1911.csv', 'data_1912-1920.csv', 
         'data_1921-1940.csv', 'data_1941-1960.csv', 'data_1961-1980.csv',
         'data_1981-2000.csv', 'data_2001-2020.csv']
frames = [pd.read_csv(f, names=['Year', 'Month', 'Day', 'TAVG', 'TMAX', 'TMIN'], usecols=[3,4,5,6,9,12]).fillna(0) for f in files ]
result = pd.concat(frames)

In [25]:
result["DOY"] = pd.to_datetime(result[["Year", "Month", "Day"]]).dt.dayofyear
result[["Year", "TAVG", "TMAX", "TMIN", "DOY"]].to_csv("1_Temperature_Kyoto.csv", index=False)

### WDC 1949-2020 的日均温度数据

In [67]:
df1 = pd.read_csv('WDC_Temp_1949-1989.csv', index_col=False)
df2 = pd.read_csv('WDC_Temp_1990-2020.csv', index_col=False)
df = pd.concat([df1, df2], sort=False)
df = df[df['STATION'] == 'USC00186350']
df['TMAX'] = df['TMAX'].replace(np.nan, 0)
df['TMIN'] = df['TMIN'].replace(np.nan, 0)

In [68]:
# 1. NaN in "TOBS" = Average(TMAX, TMIN)
col_avg = (df.loc[df['TOBS'].isna(),'TMAX'].add(df.loc[df['TOBS'].isna(),'TMIN'])) * 0.5
df.loc[df['TOBS'].isna(), 'TOBS'] = col_avg

In [69]:
# 2. 清理保留每年 1.1-4.30 的数据
df["DATE"] = pd.to_datetime(df["DATE"], format="%m/%d/%y")
df["DATE"][df["DATE"] > "2030-01-01"] = df["DATE"][df["DATE"] > "2030-01-01"] - pd.DateOffset(years=100)
result = df[df["DATE"].dt.month <= 4]

<ipython-input-69-fc5ebc7dcdce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DATE"][df["DATE"] > "2030-01-01"] = df["DATE"][df["DATE"] > "2030-01-01"] - pd.DateOffset(years=100)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [70]:
# 3. 增加column/改名/华氏度->摄氏度
result["DOY"] = result["DATE"].dt.dayofyear
result["Year"] = result["DATE"].dt.year
result.rename(columns = {"TOBS":"TAVG"}, inplace = True)
result[["TAVG", "TMAX", "TMIN"]] = result[["TAVG", "TMAX", "TMIN"]].apply(lambda x: round((x-32)* 5 / 9, 2))

<ipython-input-70-e00cb06ba78c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["DOY"] = result["DATE"].dt.dayofyear
<ipython-input-70-e00cb06ba78c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Year"] = result["DATE"].dt.year
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [71]:
result[["Year", "TAVG", "TMAX", "TMIN", "DOY"]].to_csv("1_Temperature_WDC.csv", index=False)

In [ ]:
# 记得调整 ‘TAVG’ 的位置
# pd.concat([df1, df2]).to_csv("1_Temperature_WDC.csv", header=['year', 'month', 'day', 'DOY', 'TAVG', 'TMAX', 'TMIN'], index=False)